In [1]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import OpenAIEmbedding, SentenceTransformerEncoder
from camel.types import StorageType, EmbeddingModelType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")

In [3]:
# open_ai camel_retriever
camel_retriever = AutoRetriever(
    vector_storage_local_path="local_data/embedding_storage",
    url_and_api_key=(
        "./milvus_demo.db",
        "",
        # "http://10.140.52.87:19530",  # Your Milvus connection URL
        # "",  # Your Milvus token
    ),
    storage_type=StorageType.MILVUS,
    embedding_model=OpenAIEmbedding(url=OPENAI_API_BASE, api_key=OPENAI_API_KEY),
)

In [42]:
camel_retriever = AutoRetriever(
    vector_storage_local_path="local_data/embedding_storage",
    url_and_api_key=(
        "./milvus_demo.db",
        "",
        # "http://10.140.52.87:19530",  # Your Milvus connection URL
        # "",  # Your Milvus token
    ),
    storage_type=StorageType.MILVUS,
    embedding_model=SentenceTransformerEncoder(
        model_name="intfloat/e5-large-v2"
    ),  # 1.3G
)

In [4]:
# Set one user query
query = "what's the relationship between Mistral Large 2 and Mistral AI? What kind of feature does Mistral Large 2 has?"

In [5]:
# Get related content by using vector retriever
vector_result = camel_retriever.run_vector_retriever(
    query=query,
    contents="https://mistral.ai/news/mistral-large-2407/",
)

# Show the result from vector search
print(vector_result)

{'Original Query': "what's the relationship between Mistral Large 2 and Mistral AI? What kind of feature does Mistral Large 2 has?", 'Retrieved Context': ['Large Enough\n\nToday, we are announcing Mistral Large 2, the new generation of our flagship model. Compared to its predecessor, Mistral Large 2 is significantly more capable in code generation, mathematics, and reasoning. It also provides a much stronger multilingual support, and advanced function calling capabilities.\n\nResearch\n\nJul 24, 2024\n\nMistral AI team']}


In [8]:
print(vector_result["Original Query"])

what's the relationship between Mistral Large 2 and Mistral AI? What kind of feature does Mistral Large 2 has?


In [ ]:
# Parse content from the specified URL and create knowledge graph data

# Parse content from mistral website and create knowledge graph data by using
# the Knowledge Graph Agent, store the information into graph database
uio = UnstructuredIO()

elements = uio.parse_file_or_url(
    input_path="https://mistral.ai/news/mistral-large-2407"
)
chunk_elements = uio.chunk_elements(chunk_type="chunk_by_title", elements=elements)

graph_elements = []
for chunk in chunk_elements:
    graph_elements = kg_agent.run(chunk, parse_graph_elements=True)
    n4j.add_graph_elements(graph_elements=[graph_element])
    graph_elements.append(graph_element)

In [ ]:
# Create an element from user query
query_element = uio.create_element_from_text(text=query, element_id="1")

# Let Knowledge Graph Agent extract node and relationship information from the query
ans_element = kg_agent.run(query_element, parse_graph_elements=True)

In [ ]:
# Match the entity got from query in the knowledge graph storage content
kg_result = []
for node in ans_element.nodes:
    n4j_query = f"""
MATCH (n {{id: '{node.id}'}})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {{id: '{node.id}'}})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description
"""
    result = n4j.query(query=n4j_query)
    kg_result.extend(result)

# Show the result from knowledge graph database
print(kg_result)

In [ ]:
# combine result from vector search and knowledge graph entity search
comined_results = str(vector_result) + "\n".join(kg_result)

In [ ]:
# Set up an assistant agent to answer questions based on the retrieved context:
sys_msg = BaseMessage.make_assistant_message(
    role_name="CAMEL Agent",
    content="""You are a helpful assistant to answer question,
        I will give you the Original Query and Retrieved Context,
    answer the Original Query based on the Retrieved Context.""",
)
camel_agent = ChatAgent(system_message=sys_msg, model=deepseek)

# Pass the retrieved information to agent
user_prompt = f"""
The Original Query is {query}
The Retrieved Context is {comined_results}
"""

user_msg = BaseMessage.make_user_message(role_name="CAMEL User", content=user_prompt)

# Get response
agent_response = camel_agent.step(user_msg)

print(agent_response.content)